In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

img_path = "/Users/dsz/Study/Data/test/20240710150611162_0.jpg"
img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 使用 matplotlib 显示图像
plt.imshow(img_rgb)
plt.axis('off')  # 关闭坐标轴
plt.show()

In [ ]:
color_hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]).flatten()
print(color_hist.shape)

# 绘制直方图
plt.figure(figsize=(10, 5))
plt.bar(range(len(color_hist)), color_hist, color='blue')
plt.title('Flattened Color Histogram with [8, 8, 8] Bins')
plt.xlabel('Bin Index')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
# 计算三通道的颜色直方图，使用8个分箱
hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])

# 准备数据
x = np.arange(8)
y = np.arange(8)
x, y = np.meshgrid(x, y)
z = np.zeros_like(x)

# 创建图形对象
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# 绘制每个直方图分箱的柱状图
for r in range(8):
    for g in range(8):
        for b in range(8):
            ax.bar3d(x[r, g], y[r, g], z[r, g], 1, 1, hist[r, g, b], color=(r/8, g/8, b/8, 0.6))

# 设置轴标签
ax.set_xlabel('Blue')
ax.set_ylabel('Green')
ax.set_zlabel('Red')
ax.set_title('3D Color Histogram [8, 8, 8]')

# 显示图形
plt.show()

In [ ]:

# 计算每个通道的颜色直方图
channels = ['b', 'g', 'r']
colors = ['blue', 'green', 'red']
plt.figure(figsize=(10, 5))

for i, color in enumerate(colors):
    hist = cv2.calcHist([img], [i], None, [256], [0, 256])
    plt.plot(hist, color=color, label=f'{channels[i].upper()} channel')
    plt.xlim([0, 256])

plt.title('Color Histogram')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 转换为灰度图像
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 使用 Canny 算法检测边缘
edges = cv2.Canny(gray, 100, 200)

# 显示原始图像和边缘检测结果
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(edges, cmap='gray')
plt.title('Edge Detection')
plt.axis('off')

plt.show()

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 使用 Canny 边缘检测
edges = cv2.Canny(gray, 100, 200)

# 计算边缘图像的直方图
edge_features, bins = np.histogram(edges, bins=10)
print(edge_features)

# 绘制直方图
plt.figure(figsize=(10, 5))
plt.bar(range(10), edge_features, width=0.7, color='black', align='center')
plt.xlabel('Bins')
plt.ylabel('Frequency')
plt.title('Edge Features Histogram')
plt.xticks(range(10), [f'{bins[i]:.1f}-{bins[i+1]:.1f}' for i in range(10)], rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# 转换为 HSV 颜色空间
hsv_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 直接显示 HSV 图像
plt.figure(figsize=(6, 6))
plt.imshow(hsv_image)
plt.title('HSV Image')
plt.axis('off')
plt.show()

In [ ]:
color_ranges = {
            'red': [(0, 10), (160, 180)],
            'orange': [(11, 20)],
            'yellow': [(21, 30)],  # 缩小黄色范围以减少与绿色的重叠
            'green': [(31, 80)],  # 从31开始以减少与黄色的重叠
            'cyan': [(81, 100)],
            'blue': [(101, 140)],
            'violet': [(141, 160)]
        }
index_map = ['clear', 'red', 'orange', 'yellow', 'green', 'cyan', 'blue', 'violet']
color_areas = {color: 0 for color in color_ranges}
print(color_areas)

i = 0
for color, ranges in color_ranges.items():
    mask = np.zeros(hsv_image.shape[:2], dtype=np.uint8)
    for (low, high) in ranges:
        # 创建颜色范围掩码
        single_range_mask = cv2.inRange(hsv_image, (low, 50, 50), (high, 255, 255))
        mask = cv2.bitwise_or(mask, single_range_mask)

    # 查找连通成分
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
    print(f'Number of labels: {num_labels}')
    print(f'Labels shape: {labels.shape}')
    print(f'Stats shape: {stats.shape}')
    print(f'Centroids shape: {centroids.shape}')

    # 计算最大的颜色区域
    for i in range(1, num_labels):  # 跳过背景
        area = stats[i, cv2.CC_STAT_AREA]
        if area > color_areas[color]:
            color_areas[color] = area
print(color_areas)

color_probabilities = {}
total_area = sum(color_areas.values())
for color, area in color_areas.items():
    # 计算颜色的概率（颜色面积除以所有颜色面积总和）
    color_probabilities[color] = area / total_area if total_area > 0 else 0
print(color_probabilities)

dominant_color = max(color_probabilities, key=color_probabilities.get)
print(f'Dominant color: {dominant_color}')
color_id = index_map.index(dominant_color)
print(f'Color ID: {color_id}')

In [ ]:
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, binary_img = cv2.threshold(gray_img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_img, connectivity=8, ltype=cv2.CV_32S)
output = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
colors = [tuple(np.random.randint(0, 255, size=3).tolist()) for _ in range(num_labels)]
colors[0] = (0, 0, 0)  # 通常将背景设置为黑色
for y in range(img.shape[0]):
    for x in range(img.shape[1]):
        label = labels[y, x]
        output[y, x] = colors[label]

# 使用 matplotlib 显示图像
plt.imshow(output)
plt.axis('off')  # 关闭坐标轴
plt.show()

In [ ]:
import cv2
import matplotlib.pyplot as plt

img_path = "/Users/dsz/Desktop/sonar_test.jpg"
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (2*7200, 7200))
img_rescaled = (img.astype(np.float32) * (65535.0 / 255.0)).astype(np.uint16)

# 使用 matplotlib 显示灰度图像
plt.imshow(img_rescaled, cmap='gray')
plt.axis('off')  # 关闭坐标轴
plt.show()